In [ ]:
import numpy as np
import glob
import warnings
import datetime
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from mpl_toolkits.basemap import Basemap
import os
import xarray as xr
import xskillscore as xs
import pandas as pd
import xesmf as xesmf

#### Puts all observational datasets on common 1deg x 1deg grid
#### and standardizes variable names, units, and conventions

In [ ]:
def regrid_with_nan(data,regridder,C=10.):
    data['pr'] = data['pr'] + C
    data['tas'] = data['tas'] + C
    data_rg = regridder(data)
    data_rg['pr'] = xr.where(data_rg['pr']==0.0,np.nan,data_rg['pr'])
    data_rg['tas'] = xr.where(data_rg['tas']==0.0,np.nan,data_rg['tas'])
    data_rg['pr'] = data_rg['pr'] - C
    data_rg['tas'] = data_rg['tas'] - C
    data['pr'] = data['pr'] - C
    data['tas'] = data['tas'] - C
    return data_rg

In [ ]:
diri = '/glade/work/nlybarger/data/OBS/'
latslic = slice(30,55)
lonslic = slice(225,265)

In [ ]:
gmet = xr.open_dataset(diri + 'GMET/gmet.1970-2021.e1-10.monthly.nc',drop_variables=['t_range'])
livneh = xr.open_dataset(diri + 'Livneh/livneh_prec.mon.mean.nc')
livneh = xr.merge([livneh, xr.open_dataset(diri + 'Livneh/livneh_trad.mon.mean.nc')])
cru = xr.open_dataset(diri + 'CRU/cru_ts4.06.1901.2021.pre.dat.nc',drop_variables=['stn'])
cru = xr.merge([cru, xr.open_dataset(diri + 'CRU/cru_ts4.06.1901.2021.tmp.dat.nc',drop_variables=['stn'])])
era5 = xr.open_dataset(diri + 'ERA-5/era5.mon.T.P.global_fixed.nc')
udel = xr.open_dataset(diri + 'UDel/precip.mon.total.v501.nc')
udel = xr.merge([udel, xr.open_dataset(diri + 'UDel/air.mon.mean.v501.nc')])
prism = xr.open_dataset(diri + 'PRISM/prism.mon_tot_pr.tas.nc')


In [ ]:
gmetout = xr.Dataset(
    data_vars = dict(
        pr=(['ens','time','lat','lon'], gmet['pcp'].data),
        tas=(['ens','time','lat','lon'], gmet['t_mean'].data),
        elevation=(['lat','lon'], gmet['elevation'][0,0,:,:].data),
    ),
    coords = dict(
        lon = (['lon'], gmet['longitude'][0,0,0,:].data),
        lat = (['lat'], gmet['latitude'][0,0,:,0].data),
        ens = (['ens'], gmet['ens'].data),
        time = (['time'], pd.date_range('1970-01-01','2021-12-01',freq='MS')),
    ),
)

month_length = gmetout.time.dt.days_in_month
gmetout['pr'] = gmetout['pr']*month_length
gmetout['pr'].attrs['units'] = 'mm'
gmetout['tas'].attrs['units'] = '°C'
gmetout['lon'] = gmetout['lon']+360.
gmetout = gmetout.sel(lat=latslic,lon=lonslic,drop=True)

In [ ]:
livnehout = livneh.sel(lat=latslic,lon=lonslic,drop=True)
livnehout = livnehout.rename_vars({'prec':'pr','trad':'tas'})
livnehout['tas'] = livnehout['tas'] - 273.15

In [ ]:
cruout = cru
cruout['lon'] = cruout['lon']+360.
cruout = cruout.sel(lat=latslic,lon=lonslic,drop=True)
cruout = cruout.rename_vars({'pre':'pr','tmp':'tas'})

In [ ]:
era5out = era5.sel(lat=latslic,lon=lonslic,drop=True)
month_length = era5out.time.dt.days_in_month
era5out['pr'] = era5out['pr']*month_length

In [ ]:
udelout = udel.rename_vars({'precip':'pr','air':'tas'})
udelout = udelout.reindex(lat=list(reversed(udel.lat)))
udelout = udelout.sel(lat=latslic,lon=lonslic,drop=True)
udelout['pr'] = udelout['pr']*10
udelout['pr'].attrs['units'] = 'mm'

In [ ]:
prismout = prism.sel(lat=latslic,lon=lonslic,drop=True)

In [ ]:
dummy1deg = xr.Dataset(
        data_vars = dict(
    ),
    coords = dict(
        lon = (['lon'], np.arange(236,253)),
        lat = (['lat'], np.arange(41,50)),
    ),
)


reg = xesmf.Regridder(prismout,dummy1deg,method='bilinear')
prism_r = regrid_with_nan(prismout,reg)
filo = diri + 'PRISM/1deg.prism.wconus.p.t.nc'
if os.path.exists(filo):
    os.remove(filo)
prism_r.to_netcdf(filo,mode='w')

reg = xesmf.Regridder(era5out,dummy1deg,method='bilinear')
era5_r = regrid_with_nan(era5out,reg)
filo = diri + 'ERA-5/1deg.era5.wconus.p.t.nc'
if os.path.exists(filo):
    os.remove(filo)
for it in range(len(era5_r['time'])):
    era5_r['tas'][it,:,:] = xr.where(~np.isnan(prism_r['tas'][0,:,:]),era5_r['tas'][it,:,:],np.nan)
era5_r.to_netcdf(filo,mode='w')

reg = xesmf.Regridder(gmetout,dummy1deg,method='bilinear')
gmet_r = regrid_with_nan(gmetout,reg)
gmet_r = gmet_r.mean(dim='ens')
filo = diri + 'GMET/1deg.gmetensm.wconus.p.t.nc'
if os.path.exists(filo):
    os.remove(filo)
gmet_r.to_netcdf(filo,mode='w')

reg = xesmf.Regridder(livnehout,dummy1deg,method='bilinear')
livneh_r = regrid_with_nan(livnehout,reg)
filo = diri + 'Livneh/1deg.livneh.wconus.p.t.nc'
if os.path.exists(filo):
    os.remove(filo)
livneh_r.to_netcdf(filo,mode='w')

reg = xesmf.Regridder(cruout,dummy1deg,method='bilinear')
cru_r = regrid_with_nan(cruout,reg)
filo = diri + 'CRU/1deg.cru.wconus.p.t.nc'
if os.path.exists(filo):
    os.remove(filo)
cru_r.to_netcdf(filo,mode='w')

reg = xesmf.Regridder(udelout,dummy1deg,method='bilinear')
udel_r = regrid_with_nan(udelout,reg)
filo = diri + 'UDel/1deg.udel.wconus.p.t.nc'
if os.path.exists(filo):
    os.remove(filo)
udel_r.to_netcdf(filo,mode='w')
